In [4]:
# pip install arxiv

SyntaxError: invalid syntax (3758202819.py, line 2)

In [5]:
# pip install wikipedia

  Preparing metadata (setup.py) ... done
Using legacy 'setup.py install' for wikipedia, since package 'wheel' is not installed.
  Running setup.py install for wikipedia ... done
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [14]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=500)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [8]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [11]:
tool.name

'wikipedia'

In [12]:
tool.args

{'query': {'title': 'Query', 'type': 'string'}}

In [15]:
tool.run({'query':'jason mamoa'})

'Page: Jason Momoa\nSummary: Joseph Jason Namakaeha Momoa (; born August 1, 1979) is an American actor. He made his acting debut as Jason Ioane on the syndicated action drama series Baywatch: Hawaii (1999–2001), which was followed by portrayals of Ronon Dex on the Syfy science fiction series Stargate Atlantis (2005–2009), and Khal Drogo in the first two seasons of the HBO fantasy drama series Game of Thrones (2011–2012). He went on to play the lead roles in the Discovery Channel historical drama s'

In [16]:
lined_response = tool.run('jason mamoa').split('.')[:-1]

In [21]:
'. '.join(lined_response)

'Page: Jason Momoa\nSummary: Joseph Jason Namakaeha Momoa (; born August 1, 1979) is an American actor.  He made his acting debut as Jason Ioane on the syndicated action drama series Baywatch: Hawaii (1999–2001), which was followed by portrayals of Ronon Dex on the Syfy science fiction series Stargate Atlantis (2005–2009), and Khal Drogo in the first two seasons of the HBO fantasy drama series Game of Thrones (2011–2012)'

In [23]:
tool.run('Tranzita Systems')

"Page: List of railway companies\nSummary: This is an incomplete list of the world's railway operating companies listed alphabetically by continent and country. This list includes companies operating both now and in the past.\nIn some countries, the railway operating bodies are not companies, but are government departments or authorities.\nParticularly in many European countries beginning in the late-1980s, with privatizations and the separation of the track ownership and management from running the"

In [10]:
tool.get_prompts()

[]

In [24]:
## WebabseLoader

In [25]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [26]:
web_loader = WebBaseLoader('https://www.tranzita.com/') # will read the page
doc = web_loader.load()
document = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(doc)

In [28]:
vector_db = FAISS.from_documents(document,OllamaEmbeddings())
retriever = vector_db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fc920722380>)

In [37]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever,'Tranzita Systems',
                      """Anything asked about Tranzita Systems you need to query with the retriver.
                      You need to be very specific while providing the answer""")

In [38]:
retriever_tool.name

'Tranzita Systems'

In [39]:
retriever_tool.args

{'query': {'title': 'Query',
  'description': 'query to look up in retriever',
  'type': 'string'}}

In [40]:
retriever_tool.run('tranzita systems').replace('\n','')

'Microsoft Power BIVisualize your data with Microsoft Power BICyber Security AuditSecure your applications and severs through VA/PT auditMobile FirstAll your software needs at your finger tips.Modern WebHigh performance web application for rich UX.Join us in building the best in class products with cutting edge technologies.Contact ussubmit your query and we will drop you an email.Write your email *Message *© 2021-22 Tranzita Systems. Designed with  by Tranzita Systems.ClosePreviousNextOur Trusted ClientsHow do we add value ?Start working with us on Industry 4.0 and we can provide everything\r                            in the field of IoT, Machine Learning and Big Data.Data EngineeringTransform your data on the cloud from one platform to\r                                anotherData ScienceUtilize data for Defects Detection and Predictive\r                                MaintenanceBI DashboardVisualize your data and gain key insights and trigger action\r                               

In [43]:
retriever_tool.run('Give me the headline of tranzita systems?').replace('\n','')

'Tranzita Systems - The world is one big data problemHomePages Contact  Blog  Company  About Us ServicesCareers Support   Data is what you need to do Analytics. Information is  what you need to do Business. \r                                            Transform your data into information and benefit from our expertise on designing and managing large scale data pipelines. Get StartedHow do we add value ?Start working with us on Industry 4.0 and we can provide everything in the field of IoT, Machine Learning and Big Data.Get in TouchSendYour Email has been recorded in our systems,\r              Our Team will contact You soon!\r               \r              Thank You!\r          ClosePreviousNextOur Trusted ClientsClosePreviousNextOur Trusted ClientsHow do we add value ?Start working with us on Industry 4.0 and we can provide everything\r                            in the field of IoT, Machine Learning and Big Data.Data EngineeringTransform your data on the cloud from one platform to\r

In [44]:
### Arxiv

In [45]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

In [47]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv = ArxivQueryRun(arxiv_wrapper=arxiv_wrapper)

In [ ]:
arxiv.name

'arxiv'

In [49]:
arxiv.description

'A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.'

In [50]:
combined_tools = [tool,arxiv,retriever_tool]

In [ ]:
combined_tools

In [66]:
from langchain_community.llms import Ollama

llm = Ollama(model='llama2')

In [67]:
# pip install langchainhub

In [68]:
from langchain import hub
# prompt = hub.pull('homanp/superagent')
prompt = hub.pull('hwchase17/openai-functions-agent')

In [69]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [78]:
# pip install langchain --upgrade

In [86]:
# from langchain.agents import create_tool_calling_agent, AgentExecutor
# agent = create_tool_calling_agent(llm,combined_tools,prompt)
# agent_executer = AgentExecutor(agent=agent,tools=combined_tools,verbose=True)

ImportError: cannot import name 'create_tool_calling_agent' from 'langchain.agents' (/home/bachan/Desktop/temp_dev/langchain/langchain/lib/python3.10/site-packages/langchain/agents/__init__.py)

In [87]:
from langchain.agents import create_openai_tools_agent, AgentExecutor
agent = create_openai_tools_agent(llm,combined_tools,prompt)
agent_executer = AgentExecutor(agent=agent,tools=combined_tools,verbose=True)

In [88]:
agent_executer

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [90]:
agent_executer.invoke({'input':'Tell me about Tranzita Systems'})



> Entering new AgentExecutor chain...


ValueError: Ollama call failed with status code 400. Details: {"error":"invalid options: tools"}